In [ ]:
%%sh
pip -q install mxnet autogluon ipywidgets tqdm
pip -q install s3fs # needed to load data directly from S3

In [ ]:
import pandas as pd

In [ ]:
claims_data = pd.read_csv('claims.csv')

In [ ]:
claims_data

In [ ]:
customers_data = pd.read_csv('customers.csv')

In [ ]:
customers_data

In [ ]:
dataset = claims_data.join(customers_data, how='left', on='policy_id', lsuffix='_other')

In [ ]:
dataset

In [ ]:
dataset.to_csv('dataset_autopilot.csv', index=False, header=True)

In [ ]:
import sagemaker
sess = sagemaker.Session()

prefix = 'sagemaker/DEMO-autopilot-insurance-claims'
dataset_path = sess.upload_data(path='dataset_autopilot.csv', key_prefix=prefix + '/input/training')
print(dataset_path)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor 

In [ ]:
train_data = TabularDataset(dataset_path)

# https://auto.gluon.ai/dev/api/autogluon.tabular.models.html

In [ ]:
predictor = TabularPredictor(
    label='fraud', 
    eval_metric='roc_auc'
)

In [ ]:
predictor.fit(
    train_data, 
    time_limit=7200, 
    presets='best_quality',
    excluded_model_types=['KNN','FASTAI']
)

In [ ]:
leaderboard = predictor.leaderboard()

In [ ]:
predictor.fit_summary()